In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/airbnb-listings-project-data/Airbnb Data/Listings.csv
/kaggle/input/airbnb-listings-project-data/Airbnb Data/Reviews.csv
/kaggle/input/airbnb-listings-project-data/Airbnb Data/Reviews_data_dictionary.csv
/kaggle/input/airbnb-listings-project-data/Airbnb Data/Listings_data_dictionary.csv


In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder , OneHotEncoder, MinMaxScaler, StandardScaler, RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor


In [3]:
df = pd.read_csv(
    '/kaggle/input/airbnb-listings-project-data/Airbnb Data/Listings.csv',
    encoding='ISO-8859-1',
    low_memory=False)
df


,listing_id,name,host_id,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,...,minimum_nights,maximum_nights,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable
0,281420,"Beautiful Flat in le Village Montmartre, Paris",1466919,2011-12-03,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
1,3705183,39 mÃÂ² Paris (Sacre CÃ âur),10328771,2013-11-29,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
2,4082273,"Lovely apartment with Terrace, 60m2",19252768,2014-07-31,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
3,4797344,Cosy studio (close to Eiffel tower),10668311,2013-12-17,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
4,4823489,Close to Eiffel Tower - Beautiful flat : 2 rooms,24837558,2014-12-14,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279707,38338635,Appartement T2 neuf prÃÂ¨s du tram T3a Porte ...,31161181,2015-04-13,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,1,7,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
279708,38538692,Cozy Studio in Montmartre,10294858,2013-11-27,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,7,15,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
279709,38683356,Nice and cosy mini-appartement in Paris,2238502,2012-04-27,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,6,30,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
279710,39659000,Charming apartment near Rue Saint Maur / Oberk...,38633695,2015-07-16,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,3,18,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f


In [4]:
center_lat = df['latitude'].mean()
center_lon = df['longitude'].mean()

df['distance_from_center'] = np.sqrt(
    (df['latitude'] - center_lat)**2 +
    (df['longitude'] - center_lon)**2
)


In [5]:
df['distance_x_accommodates'] = (
    df['distance_from_center'] * df['accommodates']
)

In [6]:
df.isnull().sum()

listing_id                          0
name                              175
host_id                             0
host_since                        165
host_location                     840
host_response_time             128782
host_response_rate             128782
host_acceptance_rate           113087
host_is_superhost                 165
host_total_listings_count         165
host_has_profile_pic              165
host_identity_verified            165
neighbourhood                       0
district                       242700
city                                0
latitude                            0
longitude                           0
property_type                       0
room_type                           0
accommodates                        0
bedrooms                        29435
amenities                           0
price                               0
minimum_nights                      0
maximum_nights                      0
review_scores_rating            91405
review_score

In [7]:
df['price'] = np.log1p(df['price'])

 

In [8]:
df.columns

Index(['listing_id', 'name', 'host_id', 'host_since', 'host_location',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_total_listings_count',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'district', 'city', 'latitude', 'longitude', 'property_type',
       'room_type', 'accommodates', 'bedrooms', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'instant_bookable',
       'distance_from_center', 'distance_x_accommodates'],
      dtype='object')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279712 entries, 0 to 279711
Data columns (total 35 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   listing_id                   279712 non-null  int64  
 1   name                         279537 non-null  object 
 2   host_id                      279712 non-null  int64  
 3   host_since                   279547 non-null  object 
 4   host_location                278872 non-null  object 
 5   host_response_time           150930 non-null  object 
 6   host_response_rate           150930 non-null  float64
 7   host_acceptance_rate         166625 non-null  float64
 8   host_is_superhost            279547 non-null  object 
 9   host_total_listings_count    279547 non-null  float64
 10  host_has_profile_pic         279547 non-null  object 
 11  host_identity_verified       279547 non-null  object 
 12  neighbourhood                279712 non-null  object 
 13 

In [10]:
df.isnull().sum()

listing_id                          0
name                              175
host_id                             0
host_since                        165
host_location                     840
host_response_time             128782
host_response_rate             128782
host_acceptance_rate           113087
host_is_superhost                 165
host_total_listings_count         165
host_has_profile_pic              165
host_identity_verified            165
neighbourhood                       0
district                       242700
city                                0
latitude                            0
longitude                           0
property_type                       0
room_type                           0
accommodates                        0
bedrooms                        29435
amenities                           0
price                               0
minimum_nights                      0
maximum_nights                      0
review_scores_rating            91405
review_score

In [11]:
df["price"].value_counts()

price
4.615121     6882
3.931826     6438
4.262680     6207
4.110874     6172
4.394449     6148
             ... 
7.974189        1
8.037543        1
8.892062        1
9.240287        1
11.982935       1
Name: count, Length: 5194, dtype: int64

# Fill missing values for bedrooms with median

In [12]:
# df['bedrooms'] = df['bedrooms'].fillna(df['bedrooms'].median())

In [13]:
df['name'].unique()

array(['Beautiful Flat in le Village Montmartre, Paris',
       '39 mÃ\x82Â² Paris (Sacre CÃ\x85â\x80\x9cur)',
       'Lovely apartment with Terrace, 60m2', ...,
       'Nice and cosy mini-appartement in Paris',
       'Charming apartment near Rue Saint Maur / Oberkampf',
       'Cosy apartment with view on Canal St Martin'], dtype=object)

In [14]:
df['host_location'].nunique()

7159

In [15]:
df['neighbourhood'].nunique()

660

In [16]:
# mean_price_by_neigh = df.groupby('neighbourhood')['price'].mean()
# df['neighbourhood_encoded'] = df['neighbourhood'].map(mean_price_by_neigh)

In [17]:
df['host_location'].value_counts().head(20)

host_location
Paris, Ile-de-France, France                       47794
New York, New York, United States                  24040
Rome, Lazio, Italy                                 20138
Cape Town, Western Cape, South Africa              13602
Rio de Janeiro, State of Rio de Janeiro, Brazil    13211
Sydney, New South Wales, Australia                 11350
Istanbul, Istanbul, Turkey                         10575
Mexico City, Mexico City, Mexico                    9955
FR                                                  8294
Bangkok, Thailand                                   7788
TR                                                  6257
US                                                  4751
IT                                                  4732
AU                                                  4335
BR                                                  4290
New South Wales, Australia                          3249
TH                                                  3217
Hong Kong        

# Remove district (too many missing values)

In [18]:
df = df.drop(columns=[
    
    'name','listing_id', 'host_id'], errors='ignore')

In [19]:
# df = df.sample(n=100000, random_state=42)


In [20]:
# df['host_since'] = pd.to_datetime(df['host_since'], errors='coerce')

# reference_date = pd.Timestamp('2025-01-01')

# df['host_experience_years'] = (
#     (reference_date - df['host_since']).dt.days // 365)
# df = df.drop(columns=['host_since'])

# df

In [21]:
X= df.drop(columns=['price'], errors='ignore')
y= df['price']

In [22]:
for col in X.select_dtypes(include='object').columns:
   X[col] = X[col].astype('category')

df.dtypes

host_since                      object
host_location                   object
host_response_time              object
host_response_rate             float64
host_acceptance_rate           float64
host_is_superhost               object
host_total_listings_count      float64
host_has_profile_pic            object
host_identity_verified          object
neighbourhood                   object
district                        object
city                            object
latitude                       float64
longitude                      float64
property_type                   object
room_type                       object
accommodates                     int64
bedrooms                       float64
amenities                       object
price                          float64
minimum_nights                   int64
maximum_nights                   int64
review_scores_rating           float64
review_scores_accuracy         float64
review_scores_cleanliness      float64
review_scores_checkin    

In [23]:
numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns
categorical_columns = X.select_dtypes(include=['category']).columns


In [24]:
df['property_type'].nunique()

144

In [25]:
df['city'].nunique()


10

In [26]:
df['city'].value_counts()

city
Paris             64690
New York          37012
Sydney            33630
Rome              27647
Rio de Janeiro    26615
Istanbul          24519
Mexico City       20065
Bangkok           19361
Cape Town         19086
Hong Kong          7087
Name: count, dtype: int64

In [27]:
# model = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('regressor', RandomForestRegressor(n_estimators=300,max_depth=20, random_state=42))])

In [28]:
model =RandomForestRegressor(n_estimators=50,max_depth=30,n_jobs=-1,random_state=42)

In [29]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])


categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_columns),
        ('cat', categorical_transformer, categorical_columns)])


In [30]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

In [31]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
y_predict=pipeline.predict(X_test)


In [ ]:
y_test_original = np.expm1(y_test)

y_pred_original = np.expm1(y_predict) 
 

In [ ]:
mae = mean_absolute_error(y_test_original, y_predict)
mse = mean_squared_error(y_test_original, y_predict)
rmse = mse ** 0.5
r2 = r2_score(y_test_original, y_predict)

print("MAE:", mae)
print("RMSE:", rmse)
print("R2:", r2)

In [ ]:
print("Train R2:", pipeline.score(X_train, y_train))

 

In [ ]:
# top_property_types = df['property_type'].value_counts().nlargest(20).index

# df['property_type'] = df['property_type'].apply(
#     lambda x: x if x in top_property_types else 'Other'
# )


In [ ]:
# df['property_type'].nunique()


In [ ]:
# df = df[df['price'] < df['price'].quantile(0.99)]
# top_property_types = df['property_type'].value_counts().nlargest(20).index
# df['property_type'] = df['property_type'].apply(
#     lambda x: x if x in top_property_types else 'Other')

In [ ]:
# center_lat = df['latitude'].mean()
# center_lon = df['longitude'].mean()

# df['distance_from_center'] = np.sqrt(
#     (df['latitude'] - center_lat)**2 +
#     (df['longitude'] - center_lon)**2
# )


In [ ]:
# df['distance_x_accommodates'] = (
#     df['distance_from_center'] * df['accommodates']
# )


In [ ]:
# print('distance_from_center' in df.columns)


In [ ]:
# target = 'price'

# X = df.drop(columns=[target])
# # y = df[target]
# y = np.log1p(df['price'])




In [ ]:
# numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns
# categorical_columns = X.select_dtypes(include=['category']).columns


In [ ]:
# numeric_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='median')),
#     ('scaler', StandardScaler())])


# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))])


# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numerical_columns),
#         ('cat', categorical_transformer, categorical_columns)])



In [ ]:
# model = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('regressor', RandomForestRegressor(n_estimators=50, random_state=42))])

In [ ]:
# print(numerical_columns)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42)


In [ ]:
# model.fit(X_train, y_train)

In [ ]:
# y_pred_log = model.predict(X_test)
# # y_pred = model.predict(X_test) 
# y_pred = np.expm1(y_pred_log)
# y_test_original = np.expm1(y_test)


# mae = mean_absolute_error(y_test_original, y_pred)
# mse = mean_squared_error(y_test_original, y_pred)
# rmse = mse ** 0.5
# r2 = r2_score(y_test_original, y_pred)

# print("MAE:", mae)
# print("RMSE:", rmse)
# print("R2:", r2)

In [ ]:
# df['price'].describe()
